# Data validation steps for CSV data

Import dependencies

In [2]:
import pandas as pd
import os
import sys
import datetime as dt

Load csvs

In [3]:
abs_path = 'c:/Users/antonio.alves/Documents/personal-stuff/projects/alocation-algo/operational-flexibility/'
os.chdir(abs_path)
# Get config file variables for dummy data paths
from src.config import CONFIG

data_paths = CONFIG.get('dummy_data_filepaths')

contract_types_table = pd.read_csv(data_paths.get('contract_types'), sep=';', decimal=',')
demands_table = pd.read_csv(data_paths.get('demands'), sep=';', decimal=',')
days_number_table = pd.read_csv(data_paths.get('days_numbers'), sep=';', decimal=',')
employees_table = pd.read_csv(data_paths.get('employees'), sep=';', decimal=',')
employee_hours_table = pd.read_csv(data_paths.get('employee_hours'), sep=';', decimal=',')
employee_production_lines_table = pd.read_csv(data_paths.get('employee_production_lines'), sep=';', decimal=',')
employee_shift_assignments_table = pd.read_csv(data_paths.get('employee_shift_assignments'), sep=';', decimal=',')
groups_table = pd.read_csv(data_paths.get('groups'), sep=';', decimal=',')
line_types_table = pd.read_csv(data_paths.get('line_types'), sep=';', decimal=',')
operating_types_table = pd.read_csv(data_paths.get('operating_types'), sep=';', decimal=',')
products_table = pd.read_csv(data_paths.get('products'), sep=';', decimal=',')
production_lines_table = pd.read_csv(data_paths.get('production_lines'), sep=';', decimal=',')
production_lines_stats_table = pd.read_csv(data_paths.get('production_lines_stats'), sep=';', decimal=',')
production_line_operating_type_table = pd.read_csv(data_paths.get('production_lines_operating_types'), sep=';', decimal=',')
product_production_line_assignments_table = pd.read_csv(data_paths.get('product_production_line_assignments'), sep=';', decimal=',')
product_production_line_table = pd.read_csv(data_paths.get('product_production_lines'), sep=';', decimal=',')
sections_table = pd.read_csv(data_paths.get('sections'), sep=';', decimal=',')
shifts_table = pd.read_csv(data_paths.get('shifts'), sep=';', decimal=',')
shift_types_table = pd.read_csv(data_paths.get('shift_types'), sep=';', decimal=',')


### Data transformation - Filtering dataframes to have the correct time periods

##### Explanation:
##### In this section we will filter the several dataframes needed by two lists of months and years, to only deal with values in the time periods' context.
##### This lists simulate inputed values to this stage, as user decisions.\n

##### Affected dfs: 
    - shifts_table
    - demands_table
    - days_number_table
    - employee_hours_table
    - production_lines_stats_table
    - production_line_operating_type_table

In [4]:
# Define the selected month 
months = [5, 6, 7]
year = [2024]

# Add 'month' to the dataframes that have only date
shifts_table['day'] = pd.to_datetime(shifts_table['day'], dayfirst=True)
shifts_table['month'] = shifts_table['day'].dt.month
shifts_table['year'] = shifts_table['day'].dt.year

# Filter by months and years
demands_table = demands_table[demands_table['month'].isin(months)]
demands_table = demands_table[demands_table['year'].isin(year)]

days_number_table = days_number_table[days_number_table['month'].isin(months)]
days_number_table = days_number_table[days_number_table['year'].isin(year)]

employee_hours_table = employee_hours_table[employee_hours_table['month'].isin(months)]
employee_hours_table = employee_hours_table[employee_hours_table['year'].isin(year)]

production_lines_stats_table = production_lines_stats_table[production_lines_stats_table['month'].isin(months)]
production_lines_stats_table = production_lines_stats_table[production_lines_stats_table['year'].isin(year)]

production_line_operating_type_table = production_line_operating_type_table[production_line_operating_type_table['month'].isin(months)]
production_line_operating_type_table = production_line_operating_type_table[production_line_operating_type_table['year'].isin(year)]

shifts_table = shifts_table[shifts_table['month'].isin(months)]
shifts_table = shifts_table[shifts_table['year'].isin(year)]

### Data transformation for employee table - Joining dataframes

#### In this step we merge normalized information from other tables, like names, to have more than id's

In [5]:
# Merge additional information to employee data (names)
# Contracts data
employee_df = employees_table.merge(contract_types_table, how='inner', left_on='contract_type_id', right_on='id')
employee_df = employee_df.rename(columns={
    'name_y': 'contract_type',
    'name_x': 'name', 
    'id_x': 'id'
})
employee_df = employee_df.drop(columns=['id_y'])
# Groups data
employee_df = employee_df.merge(groups_table, how='inner', left_on='group_id', right_on='id')
employee_df = employee_df.rename(columns={
    'name_y': 'group_name',
    'name_x': 'name', 
    'id_x': 'id'
})
employee_df = employee_df.drop(columns=['id_y'])
# Sections data
employee_df = employee_df.merge(sections_table, how='inner', left_on='section_id', right_on='id')
employee_df = employee_df.rename(columns={
    'name_y': 'section_name',
    'name_x': 'name', 
    'id_x': 'id'
})
employee_df = employee_df.drop(columns=['id_y'])
employee_df['group_id'] = employee_df['group_id'].astype(int)
print(f"emloyees_table columns: {employees_table.columns}")
print(f"employee_df columns: {employee_df.columns}")
print(f"emloyees_table row number: {len(employees_table)}")
print(f"emloyee_df row number: {len(employee_df)}")

emloyees_table columns: Index(['id', 'name', 'contract_type_id', 'group_id', 'employee_type',
       'capacity_contribution', 'bank_hours', 'section_id'],
      dtype='object')
employee_df columns: Index(['id', 'name', 'contract_type_id', 'group_id', 'employee_type',
       'capacity_contribution', 'bank_hours', 'section_id', 'contract_type',
       'group_name', 'section_name'],
      dtype='object')
emloyees_table row number: 100
emloyee_df row number: 48


### Data transformation for shifts table - joining dataframes

##### The same as before but now for the shift information for employees

In [6]:
# Get the calendars for all employees
shift_df = shifts_table.merge(shift_types_table, how='inner', left_on='shifttype_id', right_on='id')
shift_df = shift_df.rename(columns={
    'id_x': 'id'
})
shift_df = shift_df.drop(columns=['name', 'id_y', 'shifttype_id'])
shift_df = shift_df.merge(groups_table, how='inner', left_on='group_id', right_on='id')
shift_df = shift_df.rename(columns={
    'id_x': 'id',
    'name_y': 'group_name',
    'name_x': 'name'
})
shift_df = shift_df.drop(columns=['id_y'])
employee_groups_df = employee_df.drop(columns=['name', 'contract_type', 'employee_type', 'capacity_contribution', 'bank_hours', 'section_id', 'contract_type', 'section_name', 'group_name'])
employee_groups_df = employee_groups_df.rename(columns={'id': 'employee_id'})
employee_groups_df = employee_groups_df.merge(shift_df, how='inner', left_on='group_id', right_on='group_id')
employee_groups_df = employee_groups_df.drop(columns=['id'])
print(f"shifts_table columns: {shifts_table.columns}")
print(f"employee_groups_df columns: {employee_groups_df.columns}")
print(employee_groups_df)

shifts_table columns: Index(['id', 'group_id', 'week_number', 'week_day', 'day', 'shifttype_id',
       'month', 'year'],
      dtype='object')
employee_groups_df columns: Index(['employee_id', 'contract_type_id', 'group_id', 'week_number',
       'week_day', 'day', 'month', 'year', 'value', 'timestamp_in',
       'timestamp_out', 'name'],
      dtype='object')
      employee_id  contract_type_id  group_id  week_number  week_day  \
0               2                 1         1           18         3   
1               2                 1         1           18         4   
2               2                 1         1           18         5   
3               2                 1         1           18         6   
4               2                 1         1           19         7   
...           ...               ...       ...          ...       ...   
2019           41                 1         1           30         6   
2020           41                 1         1           31  

### Data transformation for production lines - joining dataframes

##### Still the same but now with production lines and products

In [7]:
product_production_line_df = product_production_line_table.merge(products_table, how='inner', left_on='product_id', right_on='id')
product_production_line_df = product_production_line_df.rename(columns={
    'id_x': 'id',
    'name': 'product_name',
    'diameter': 'product_diameter',
    'height': 'product_height',
    'material': 'product_material'
})
product_production_line_df = product_production_line_df.drop(columns=['id_y'])

product_production_line_df = product_production_line_df.merge(production_lines_table, how='inner', left_on='production_line_id', right_on='id')
product_production_line_df = product_production_line_df.rename(columns={
    'id_x': 'id',
    'name': 'production_line_name'
})
product_production_line_df = product_production_line_df.drop(columns=['id_y'])

product_production_line_df = product_production_line_df.merge(production_lines_stats_table, how='inner', left_on='id', right_on='product_production_line_id')
product_production_line_df = product_production_line_df.rename(columns={
    'id_x': 'id',
    'name': 'production_line_name'
})
product_production_line_df = product_production_line_df.drop(columns=['id_y'])
product_production_line_df = product_production_line_df.drop(columns=['product_production_line_id'])
print(product_production_line_df)

     id  product_id  production_line_id  preferred_level  active product_name  \
0     1           1                  14             1.00       1    AE 45x118   
1     1           1                  14             1.00       1    AE 45x118   
2     1           1                  14             1.00       1    AE 45x118   
3     2           2                  14             1.00       1    AE 45x140   
4     2           2                  14             1.00       1    AE 45x140   
..   ..         ...                 ...              ...     ...          ...   
106  36          14                  14             0.01       1    AE 52x234   
107  36          14                  14             0.01       1    AE 52x234   
108  37          15                  14             0.01       1    AE 52x244   
109  37          15                  14             0.01       1    AE 52x244   
110  37          15                  14             0.01       1    AE 52x244   

     product_diameter  prod

### Data transformation for production lines - joining dataframes - add operating types information to production_line_operating_type_df

In [8]:
# Add calculations to determine the various values for different production lines based on the operating types
# Initialize a dict to store the values for the amount of possible shifts each month
shifts_amount = {
    'id': [i for i in range(1, 2*len(months) + 1)],
    'operating_type_id': [i for i in range(1,3) for _ in range(len(months))], # range(1,3) is to repeat 1 and 2 len(months) times
    'month': months * 2,                                                      # *2: because we have two shifts
    'year': year * len(months) * 2                                            # *2: because we have two shifts
}
#print(shifts_amount)
shifts_amount_df = pd.DataFrame(shifts_amount)

merged_df = pd.merge(
    shifts_amount_df,
    days_number_table,
    on=['month', 'year'],
    how='left',
    suffixes=('', '_days')
)

# Step 2: Create the shifts_amount column using numpy.where for conditional logic
import numpy as np

merged_df['shifts_amount'] = np.where(
    merged_df['operating_type_id'] == 1,
    merged_df['working_days'] * 3,
    merged_df['working_days'] * 3 + merged_df['saturdays'] * 3 + merged_df['sundays'] * 2
)
shifts_amount_df = pd.DataFrame(merged_df[['id', 'operating_type_id', 'month', 'year', 'shifts_amount']])

prodlines = list(set((production_line_operating_type_table['production_line_id'].to_list())))
max_id = production_line_operating_type_table['id'].max()
production_line_operating_type_df = pd.DataFrame(production_line_operating_type_table)

# First approach, just to be sure what needs to happen
#for year1 in year:
#    for month in months:
#        for prodline in prodlines:
#            query_plot_df = production_line_operating_type_df.query(f"month == {month} and year == {year1} and production_line_id == {prodline} and operating_type_id == {3}")
#            number_of_shifts_3 = shifts_amount_df.query(f"month == {month} and year == {year1} and operating_type_id == {1}")['shifts_amount'].values[0].item()
#            stopage_shifts = production_line_operating_type_df.query(f"month == {month} and year == {year1} and operating_type_id == {3}")['number_of_shifts'].values[0].item()
#            result1 = number_of_shifts_3 - stopage_shifts
#            #production_line_operating_type_table.loc[(production_line_operating_type_table['month'] == month) & (production_line_operating_type_table['year'] == year1) & (production_line_operating_type_table['operating_type_id'] == 1), 'number_of_shifts'] = result1
#            
#            number_of_shifts_4 = shifts_amount_df.query(f"month == {month} and year == {year1} and operating_type_id == {2}")['shifts_amount'].values[0].item()
#            result2 = number_of_shifts_4 - stopage_shifts
#            #production_line_operating_type_table.loc[(production_line_operating_type_table['month'] == month) & (production_line_operating_type_table['year'] == year1) & (production_line_operating_type_table['operating_type_id'] == 2), 'number_of_shifts'] = result2
#            
#            max_id += 1
#            dict_to_append = {
#                'id': max_id,
#                'production_line_id': prodline,
#                'operating_type_id': 1,
#                'month': month,
#                'year': year,
#                'number_of_shifts': result1
#            }
#            new_row = pd.DataFrame(dict_to_append, index=[0])
#            production_line_operating_type_df = pd.concat([production_line_operating_type_df, new_row], ignore_index=True)
#            max_id += 1
#            dict_to_append = {
#                'id': max_id,
#                'production_line_id': prodline,
#                'operating_type_id': 2,
#                'month': month,
#                'year': year,
#                'number_of_shifts': result2
#            }
#            new_row = pd.DataFrame(dict_to_append, index=[0])
#            production_line_operating_type_df = pd.concat([production_line_operating_type_df, new_row], ignore_index=True)
            
# 1. Get the unique production lines, years, and months we're working with
production_lines = production_line_operating_type_df['production_line_id'].unique()
target_months = months  # [1, 2, 3]
target_years = year     # [2024]

# 2. Create a DataFrame with all combinations we need to ensure exist
from itertools import product
all_combinations = pd.DataFrame(list(product(
    production_lines, 
    [1, 2, 3],  # operating_type_ids
    target_months,
    target_years
)), columns=['production_line_id', 'operating_type_id', 'month', 'year'])

# 3. Filter for just the operating_type_id 1 and 2 combinations that we need to calculate
missing_combinations = all_combinations[all_combinations['operating_type_id'].isin([1, 2])]

# 4. Create a mapping from type 3 data for stoppage_shifts
type3_data = production_line_operating_type_df[
    (production_line_operating_type_df['operating_type_id'] == 3)
].copy().rename(columns={'number_of_shifts': 'stoppage_shifts'})

# 5. Merge to get stoppage_shifts for each production line, month, year
result = missing_combinations.merge(
    type3_data[['production_line_id', 'month', 'year', 'stoppage_shifts']],
    on=['production_line_id', 'month', 'year'],
    how='left'
)

# 6. Create a mapping from shifts_amount_df
shifts_data = shifts_amount_df[
    (shifts_amount_df['operating_type_id'].isin([1, 2])) &
    (shifts_amount_df['month'].isin(target_months)) &
    (shifts_amount_df['year'].isin(target_years))
].copy()[['operating_type_id', 'month', 'year', 'shifts_amount']]

# 7. Merge to get shifts_amount based on operating_type_id, month, year
result = result.merge(
    shifts_data,
    on=['operating_type_id', 'month', 'year'],
    how='left'
)

# 8. Calculate the number_of_shifts as shifts_amount - stoppage_shifts
result['number_of_shifts'] = result['shifts_amount'] - result['stoppage_shifts']

# 9. Clean up and prepare for merging
result.drop(['stoppage_shifts', 'shifts_amount'], axis=1, inplace=True)

# 10. Generate new IDs
next_id = max_id + 1
result['id'] = range(next_id, next_id + len(result))
max_id = next_id + len(result) - 1

# 11. Get the combinations that aren't already in the DataFrame
existing_combos = set(map(tuple, production_line_operating_type_df[
    ['production_line_id', 'operating_type_id', 'month', 'year']
].values))

result = result[~result.apply(
    lambda row: (row['production_line_id'], row['operating_type_id'], row['month'], row['year']) in existing_combos, 
    axis=1
)]

# 12. Append only the new combinations to the original DataFrame
production_line_operating_type_df = pd.concat([production_line_operating_type_df, result], ignore_index=True)
production_line_operating_type_df


,id,production_line_id,operating_type_id,month,year,number_of_shifts
0,5,1,3,5,2024,0.0
1,6,1,3,6,2024,0.0
2,7,1,3,7,2024,0.0
3,17,2,3,5,2024,2.0
4,18,2,3,6,2024,2.0
...,...,...,...,...,...,...
220,441,25,1,6,2024,44.0
221,442,25,1,7,2024,59.0
222,443,25,2,5,2024,73.0
223,444,25,2,6,2024,69.0


### Data transformation for calculating horas reais for each production line

In [9]:
# Store the df in another variable
aux_df = production_line_operating_type_df.copy()

aux_df['theoretical_hours_amount'] = np.where(
    aux_df['operating_type_id'] == 3,
    0,
    aux_df['number_of_shifts'].apply(lambda x: x*8)
)
aux_df = pd.DataFrame(aux_df[['id', 'production_line_id', 'operating_type_id', 'month', 'year', 'number_of_shifts', 'theoretical_hours_amount']])
production_line_operating_type_df = aux_df

production_line_operating_type_df

,id,production_line_id,operating_type_id,month,year,number_of_shifts,theoretical_hours_amount
0,5,1,3,5,2024,0.0,0.0
1,6,1,3,6,2024,0.0,0.0
2,7,1,3,7,2024,0.0,0.0
3,17,2,3,5,2024,2.0,0.0
4,18,2,3,6,2024,2.0,0.0
...,...,...,...,...,...,...,...
220,441,25,1,6,2024,44.0,352.0
221,442,25,1,7,2024,59.0,472.0
222,443,25,2,5,2024,73.0,584.0
223,444,25,2,6,2024,69.0,552.0


### Data transformation to add demand to product production lines connection table

In [10]:
# Save as df
demands_df = demands_table.copy()

# Merge data to product production line
merged_df = pd.merge(
    product_production_line_df,
    demands_df,
    on=['product_id', 'month', 'year'],
    how='left',
    suffixes=('', '_from_df2')
)
merged_df = merged_df.drop(columns=['id_from_df2'])
merged_df = merged_df.rename(columns={'value': 'demand'})
product_production_line_df = merged_df.copy()
print(f"product_production_line_df: {product_production_line_df}")



product_production_line_df:      id  product_id  production_line_id  preferred_level  active product_name  \
0     1           1                  14             1.00       1    AE 45x118   
1     1           1                  14             1.00       1    AE 45x118   
2     1           1                  14             1.00       1    AE 45x118   
3     2           2                  14             1.00       1    AE 45x140   
4     2           2                  14             1.00       1    AE 45x140   
..   ..         ...                 ...              ...     ...          ...   
106  36          14                  14             0.01       1    AE 52x234   
107  36          14                  14             0.01       1    AE 52x234   
108  37          15                  14             0.01       1    AE 52x244   
109  37          15                  14             0.01       1    AE 52x244   
110  37          15                  14             0.01       1    AE 52x244   


### Data transformation to check how many hours it would be needed to operate that demand

In [11]:
import pandas as pd
import numpy as np

# Step 1: Calculate needed_hours in the original dataframe
aux_df = product_production_line_df.copy()
aux_df['needed_hours'] = (aux_df['demand'] * 1000) / (aux_df['production_rate'] * aux_df['oee'])

# Step 2: Create a dataframe with the sum of needed_hours for the specific combination you need
# This will ensure real_hours_amount is the same for each month-year-production_line-diameter-section combination
aggregation_df = aux_df.groupby(['production_line_id', 'product_diameter', 'section_id', 'month', 'year'])['needed_hours'].sum().reset_index()
aggregation_df = aggregation_df.rename(columns={'needed_hours': 'real_hours_amount'})

# Step 3: Merge this back to the original data
result = aux_df.merge(
    aggregation_df,
    on=['production_line_id', 'product_diameter', 'section_id', 'month', 'year'],
    how='left'
)

# Step 4: Merge with operating type info
result2 = result.merge(
    production_line_operating_type_df[['production_line_id', 'operating_type_id', 'month', 'year', 'theoretical_hours_amount']],
    on=['production_line_id', 'month', 'year'],
    how='left'
)

# Step 5: Handle NaN values that might occur after the merge
result2['theoretical_hours_amount'] = result2['theoretical_hours_amount'].fillna(0)
result2['operating_type_id'] = result2['operating_type_id'].fillna(0)

# Step 6: Create the validity flag
result2['valid_operating_type'] = (
    (result2['real_hours_amount'] <= result2['theoretical_hours_amount']) | 
    (result2['operating_type_id'] == 3)
)

# Verify we didn't lose any products
product_count_original = aux_df['product_id'].nunique()
product_count_result = result2['product_id'].nunique()
print(f"Number of unique products in original data: {product_count_original}")
print(f"Number of unique products in result: {product_count_result}")
# Verify that real_hours_amount is consistent across the specified combinations
check_consistency = result2.groupby(['production_line_id', 'product_diameter', 'section_id', 'month', 'year'])['real_hours_amount'].nunique()
print("\nNumber of unique real_hours_amount values per combination:")
print(check_consistency.value_counts())

Number of unique products in original data: 15
Number of unique products in result: 15

Number of unique real_hours_amount values per combination:
real_hours_amount
1    21
Name: count, dtype: int64


In [12]:
# add to result2 delta(real_hours_amount - theoretical_hours_amount)

result2['delta_hours_amount'] = np.where(
    (result2['operating_type_id'] == 3),
    0,
    result2['theoretical_hours_amount'] - result2['real_hours_amount']
)

product_production_agg_df = result2.copy()
print(product_production_agg_df.columns)

Index(['id', 'product_id', 'production_line_id', 'preferred_level', 'active',
       'product_name', 'product_diameter', 'product_height',
       'product_material', 'production_line_name', 'employees_needed',
       'section_id', 'month', 'year', 'production_rate', 'oee', 'demand',
       'needed_hours', 'real_hours_amount', 'operating_type_id',
       'theoretical_hours_amount', 'valid_operating_type',
       'delta_hours_amount'],
      dtype='object')


In [14]:
printable_df = product_production_agg_df.copy()
printable_df[['product_id', 'production_line_id', 'product_name', 'production_line_name', 'month', 'year', 'real_hours_amount', 'operating_type_id', 'theoretical_hours_amount', 'delta_hours_amount']]

,product_id,production_line_id,product_name,production_line_name,month,year,real_hours_amount,operating_type_id,theoretical_hours_amount,delta_hours_amount
0,1,14,AE 45x118,L16,5,2024,666.013205,3,0.0,0.000000
1,1,14,AE 45x118,L16,5,2024,666.013205,1,476.0,-190.013205
2,1,14,AE 45x118,L16,5,2024,666.013205,2,636.0,-30.013205
3,1,14,AE 45x118,L16,6,2024,429.886551,3,0.0,0.000000
4,1,14,AE 45x118,L16,6,2024,429.886551,1,404.0,-25.886551
...,...,...,...,...,...,...,...,...,...,...
328,15,14,AE 52x244,L16,6,2024,852.318224,1,404.0,-448.318224
329,15,14,AE 52x244,L16,6,2024,852.318224,2,604.0,-248.318224
330,15,14,AE 52x244,L16,7,2024,1045.499803,3,0.0,0.000000
331,15,14,AE 52x244,L16,7,2024,1045.499803,1,524.0,-521.499803


### Data transformation for employee and production lines

In [12]:
print(f"production_line_operating_type_df: {production_line_operating_type_df}")
print(f"product_production_line_df: {product_production_line_df}")

production_line_operating_type_df:       id  production_line_id  operating_type_id  month  year  \
0      5                   1                  3      5  2024   
1      6                   1                  3      6  2024   
2      7                   1                  3      7  2024   
3     17                   2                  3      5  2024   
4     18                   2                  3      6  2024   
..   ...                 ...                ...    ...   ...   
220  441                  25                  1      6  2024   
221  442                  25                  1      7  2024   
222  443                  25                  2      5  2024   
223  444                  25                  2      6  2024   
224  445                  25                  2      7  2024   

     number_of_shifts  theoretical_hours_amount  
0                 0.0                       0.0  
1                 0.0                       0.0  
2                 0.0                       0.

### Drop NA's (later save the removed indexes')

In [ ]:
employee_df.dropna()
employee_groups_df.dropna()
product_production_line_df.dropna()
print(employee_hours_table.columns)

### Employee capacity contribution and bank hours

In [ ]:
employee_hours_table['total_hours_leave'] = employee_hours_table['leave_fte'] + employee_hours_table['loan_fte'] + employee_hours_table['holidays_fte'] + employee_hours_table['training_hours']
print(employee_hours_table)

### Columns

In [13]:
print(f"Employee df columns: {employee_df.columns}")
print(f"Employee hours df columns: {employee_hours_table.columns}")
print(f"Employee shifts df columns: {employee_groups_df.columns}")
print(f"Employee production lines df columns: {employee_production_lines_table.columns}")
print(f"Product production lines df columns: {product_production_line_df.columns}")
print(f"Production lines operating types df columns: {production_line_operating_type_df.columns}")
print(f"Product production aggregate information columns: {product_production_agg_df.columns}")
print(f"Demand df columns: {demands_table.columns}")

Employee df columns: Index(['id', 'name', 'contract_type_id', 'group_id', 'employee_type',
       'capacity_contribution', 'bank_hours', 'section_id', 'contract_type',
       'group_name', 'section_name'],
      dtype='object')
Employee hours df columns: Index(['id', 'employee_id', 'month', 'year', 'leave_fte', 'loan_fte',
       'holidays_fte', 'training_hours'],
      dtype='object')
Employee shifts df columns: Index(['employee_id', 'contract_type_id', 'group_id', 'week_number',
       'week_day', 'day', 'month', 'year', 'value', 'timestamp_in',
       'timestamp_out', 'name'],
      dtype='object')
Employee production lines df columns: Index(['id', 'employee_id', 'production_line_id', 'head_operator',
       'competency_level'],
      dtype='object')
Product production lines df columns: Index(['id', 'product_id', 'production_line_id', 'preferred_level', 'active',
       'product_name', 'product_diameter', 'product_height',
       'product_material', 'production_line_name', 'employee

In [15]:
print(f"Product production lines df columns: {product_production_line_df.columns}")
print(f"Production lines operating types df columns: {production_line_operating_type_df.columns}")
print(f"Product production aggregate information columns: {product_production_agg_df.columns}")

print(product_production_line_df.columns.isin(product_production_agg_df.columns))
print(production_line_operating_type_df.columns.isin(product_production_agg_df.columns))

Product production lines df columns: Index(['id', 'product_id', 'production_line_id', 'preferred_level', 'active',
       'product_name', 'product_diameter', 'product_height',
       'product_material', 'production_line_name', 'employees_needed',
       'section_id', 'month', 'year', 'production_rate', 'oee', 'demand'],
      dtype='object')
Production lines operating types df columns: Index(['id', 'production_line_id', 'operating_type_id', 'month', 'year',
       'number_of_shifts', 'theoretical_hours_amount'],
      dtype='object')
Product production aggregate information columns: Index(['id', 'product_id', 'production_line_id', 'preferred_level', 'active',
       'product_name', 'product_diameter', 'product_height',
       'product_material', 'production_line_name', 'employees_needed',
       'section_id', 'month', 'year', 'production_rate', 'oee', 'demand',
       'needed_hours', 'real_hours_amount', 'operating_type_id',
       'theoretical_hours_amount', 'valid_operating_type',
  